Import Libraries

In [ ]:
import keras
from keras import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pickle
import pandas as pd
import numpy as np
import random
from PIL import Image, ImageOps


Clone the traffic sign image and label data

In [ ]:
!git clone https://bitbucket.org/jadslim/german-traffic-signs

Read the descriptions of the labels for the various signs

In [ ]:
data = pd.read_csv('german-traffic-signs/signnames.csv')
data

Import the train, test and validation data as pickle files

In [5]:
with open('german-traffic-signs/train.p',mode='rb') as training:
  train = pickle.load(training)

with open('german-traffic-signs/valid.p',mode='rb') as validation:
  valid = pickle.load(validation)

with open('german-traffic-signs/test.p', mode='rb') as testing:
  test = pickle.load(testing)

Split the images and labels in each dataset

In [6]:
X_train, y_train = train['features'], train['labels']
X_validation, y_validation = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

(This bit is to save some test images as jpegs for an interactive web app demonstration of the model)

In [ ]:
for i in np.arange(15):
  print(i)
  im = Image.fromarray(X_test[i])
  im.save("/content/drive/My Drive/Colab Notebooks/Traffic/german-traffic-signs/image/sign_{}.jpg".format(i))

Sample:

In [ ]:
index = np.random.randint(1,len(X_train))
plt.imshow(X_train[index])
print("Image label = {}".format(y_train[index]))

Shuffle the training data

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train,y_train)

Preprocessing function to make the image grayscale, equalise the image histogram and normalise the image

In [ ]:
def preprocessing(img):
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img = cv2.equalizeHist(img)
  img = img/255
  return img

Preprocess the datasets with the preprocessing function + resize them to format them for the model

In [ ]:
X_train_processed = np.array(list(map(preprocessing,X_train)))
X_validation_processed = np.array(list(map(preprocessing,X_validation)))
X_test_processed = np.array(list(map(preprocessing,X_test)))

In [ ]:
X_train_processed = X_train_processed.reshape(X_train.shape[0],32,32,1)
X_validation_processed = X_validation_processed.reshape(X_validation.shape[0],32,32,1)
X_test_processed = X_test_processed.reshape(X_test.shape[0],32,32,1)

In [ ]:
random_int = np.random.randint(1,len(X_train))
plt.imshow(X_train_processed[random_int].squeeze(),cmap='gray')
plt.figure()
plt.imshow(X_train[random_int].squeeze(),cmap='gray')

Define, compile, fit and evaluate the model

In [ ]:
model = Sequential()
model.add(Conv2D(32,(5,5),activation='relu',input_shape=[32,32,1]))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(5,5),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(256,activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(43,activation='softmax'))

model.summary()


In [ ]:
model.compile(Adam(lr=0.00015),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_processed,
                    y_train,
                    batch_size = 512, 
                    epochs = 50, 
                    verbose=1, 
                    validation_data = (X_validation_processed,y_validation))

In [ ]:
score = model.evaluate(X_test_processed,y_test)
print('Test accuracy',score[1])

In [ ]:
history.history.keys()

Sample the model's functionality

In [ ]:
prediction = model.predict_classes(X_test_processed)
y_true_label = y_test

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_true_label,prediction)
plt.figure(figsize=(20,20))
sns.heatmap(matrix,annot=True)

In [ ]:
L=6
W=6
fig, axes = plt.subplots(L,W,figsize=(24,24))
axes = axes.ravel()
for i in range(0,L*W):
  axes[i].imshow(X_test[i])
  axes[i].set_title('Prediction={}\n True={}'.format(data.SignName[prediction[i]],data.SignName[y_true_label[i]]))
  axes[i].axis('off')
plt.subplots_adjust(wspace=2)  

Save the model to an hd5 file

In [ ]:
model.save('traffic.h5')